In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from reproject import reproject_interp
from astropy.visualization import make_lupton_rgb
from ImagingHelpers import Image, rgb_scale as rgbscale

In [2]:
# Look for files in the current directory
path = './'

In [3]:
B = Image(path+'NM20in-20200405-031953-M37-B.fts')
V = Image(path+'NM20in-20200405-040513-M37-V.fts')
R = Image(path+'NM20in-20200405-042428-M37-R.fts')

Make an image with just the data as it comes to show the misalignment.

In [23]:
plt.figure()
#plt.imshow(B.data)
plt.plot(B.data[2048,:])
plt.show()

In [4]:
Bu = rgbscale(B.data, log=True, vmin=3,vmax=3.4)
Vu = rgbscale(V.data, log=True, vmin=3,vmax=3.4)
Ru = rgbscale(R.data, log=True, vmin=3,vmax=3.4)

In [24]:
rgbu = np.stack((Ru,Vu,Bu), axis=2)

In [25]:
plt.figure()
plt.imshow(rgbu)
plt.show()

Now do reprojection, relative to B.

In [4]:
fits.writeto('M37_B_rp.fits', B.data, header=B.header, overwrite=True)

In [5]:
%%time
try:
    Vr = fits.open('M37_V_rp.fits')[0].data
    Vfoot = fits.open('M37_Vfoot_rp.fits')[0].data
except:
    Vr, Vfoot = reproject_interp(V.hdu, B.header)
    fits.writeto('M37_V_rp.fits', Vr, header=B.header)
    fits.writeto('M37_Vfoot_rp.fits', Vfoot, header=B.header)

CPU times: user 3.88 ms, sys: 1.93 ms, total: 5.81 ms
Wall time: 28.3 ms


In [6]:
%%time
try:
    Rr = fits.open('M37_R_rp.fits')[0].data
    Rfoot = fits.open('M37_Rfoot_rp.fits')[0].data
except:
    Rr, Rfoot = reproject_interp(R.hdu, B.header)
    fits.writeto('M37_R_rp.fits', Rr, header=B.header)
    fits.writeto('M37_Rfoot_rp.fits', Rfoot, header=B.header)

CPU times: user 3.97 ms, sys: 2.07 ms, total: 6.04 ms
Wall time: 25.5 ms


In [29]:
plt.figure()
plt.imshow(Rfoot)
plt.colorbar()
plt.show()

In [11]:
plt.figure()
plt.imshow(Rr, vmax=2000)
plt.colorbar()
plt.show()

In [13]:
Blsc = rgbscale(B.data, log=True, vmin=3,vmax=3.2)
Vlsc = rgbscale(Vr, log=True, vmin=3,vmax=3.2)
Rlsc = rgbscale(Rr, log=True, vmin=3,vmax=3.2)

/Users/jaguirre/Documents/Teaching/ASTR250/ASTR250CourseMaterial/ImagingHelpers.py:34: RuntimeWarning: invalid value encountered in less_equal
  scaled[scaled <= vmin] = vmin
/Users/jaguirre/Documents/Teaching/ASTR250/ASTR250CourseMaterial/ImagingHelpers.py:38: RuntimeWarning: invalid value encountered in greater_equal
  scaled[scaled >= vmax] = vmax


In [14]:
rgb = np.stack((Rlsc,Vlsc,Blsc), axis=2)

In [18]:
plt.figure()
plt.subplot(projection = B.wcs)
plt.imshow(rgb)
#plt.colorbar(shrink=0.8)
plt.show()